In [1]:
import matplotlib
%matplotlib tk
#matplotlib.use('Agg')

%load_ext autoreload
%autoreload 2

%autosave 180

import matplotlib.pyplot as plt
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

import numpy as np
import os

import sys
sys.path.append('..')

# visualize results module
from visualize import visualize_svm as Visualize



Autosaving every 180 seconds


<ipython-input-1-eeb477d8c4a5>:11: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


In [2]:
###########################################
############ LOAD LOCANMF COMPS ###########
###########################################
root_dir = '/media/cat/4TBSSD/yuki/'
animal_id = 'IA1'
session_id = 'IA1pm_Feb2_30Hz'
session_id = 'IA1pm_Feb16_30Hz'
#session_id = 'IA1pm_Feb3_30Hz'

# animal_id = 'AQ2'
# session_id = 'AQ2am_Apr1_Week4_30Hz'


sessions = Visualize.get_sessions(root_dir,
                                  animal_id,
                                  'all')
print ("# sessions: ", len(sessions))

import pylab
import imageio
filename = '/media/cat/4TBSSD/yuki/IA1/vids/prestroke/IA1pm_Feb16_30Hz.mp4'
vid = imageio.get_reader(filename,  'ffmpeg')
print (vid.count_frames())
# 
video = np.zeros((vid.count_frames(),360,640),'uint8')
for i, im in enumerate(vid):
    
    video[i]=im[:,:,1]
    if i%1000==0:
        print('Mean of frame %i is %1.1f' % (i, im.mean()))
   


# sessions:  71
19954
Mean of frame 0 is 40.4
Mean of frame 1000 is 112.7
Mean of frame 2000 is 112.0
Mean of frame 3000 is 112.6
Mean of frame 4000 is 111.2
Mean of frame 5000 is 115.3
Mean of frame 6000 is 118.5
Mean of frame 7000 is 117.6
Mean of frame 8000 is 117.8
Mean of frame 9000 is 118.3
Mean of frame 10000 is 117.5
Mean of frame 11000 is 119.0
Mean of frame 12000 is 118.2
Mean of frame 13000 is 117.0
Mean of frame 14000 is 119.8
Mean of frame 15000 is 117.7


KeyboardInterrupt: 

In [3]:
# 
print (video.shape)

diff = video[1:]-video[:-1]

print (diff.shape)

sums = diff.reshape(diff.shape[0],-1).mean(1)





(19954, 360, 640)
(19953, 360, 640)


KeyboardInterrupt: 

In [4]:

root_dir = '/media/cat/4TBSSD/yuki/'
animal_id = 'IA1'
#session_id = 'IA1pm_Feb2_30Hz'
session_id = 'IA1pm_Feb16_30Hz'
#session_id = 'IA1pm_Feb3_30Hz'

# animal_id = 'AQ2'
# session_id = 'AQ2am_Apr1_Week4_30Hz'


sessions = Visualize.get_sessions(root_dir,
                                  animal_id,
                                  'all')
print ("# sessions: ", len(sessions))

# look at specific trials and neural activity
fname = os.path.join(
    root_dir, 
    animal_id,
    'tif_files',
    session_id,
    session_id+'_body_and_nonreward_lockout_0secLockout_[]bodyfeats.npz')

data = np.load(fname, allow_pickle=True)
names_behaviors = data['names']
print ("names: ", names_behaviors)
trials = data['trials']
print (trials.shape)
random = data['random']
behavior_times = data['behaviors']
print (behavior_times.shape)
# t = np.arange(trials.shape[1])/30.-30
# plt.plot(t,trials[2,:,:])
# plt.show()

shift = np.loadtxt(os.path.join(
                            root_dir, 
                            animal_id,
                            'tif_files',
                            session_id,
                            'shift.txt'))
                   
print ("Shifts: ", shift)

blue_light = np.load(os.path.join(
                            root_dir, 
                            animal_id,
                            'tif_files',
                            session_id,
                            'blue_light_frame_triggers.npz'))

start_blue = blue_light['start_blue']/30
print ("start blue: ", start_blue)

fname = os.path.join(
                root_dir, 
                animal_id,
                'tif_files',
                session_id,
                session_id+'_locanmf_wholestack.npz')

data = np.load(fname, allow_pickle=True)

names = data['areanames_area']
areas = data['areas']
names2 = data['names']
A = data["A_reshape"]
print (A.shape)
whole_stack = data["whole_stack"]
print (whole_stack.shape)

clrs = ['pink','blue','lightblue','slategrey','black','magenta','khaki']
ctr_clrs =0
#
for k in range(len(names2)):
    print (k, names2[k])

area_ids = [12, 8, 13]  # left limb, left M1, left visual
ca = whole_stack[area_ids]
print (ca.shape)

#
plt.figure()

# plot video movements
t=np.arange(sums.shape[0])/30
plt.plot(t,sums, label='video diff')

t = np.arange(ca.shape[1])/30.

# plot calcium activity
for k in range(ca.shape[0]):
    temp = ca[k]
    temp = temp - np.median(temp) + k*350
    plt.plot(t,temp, 
             c=clrs[ctr_clrs],
             linewidth=2,
             label = names2[area_ids][k])

    ctr_clrs+=1
    
# plot rewarded lever pulls
temp = behavior_times[0]/30.
plt.axvline(x=temp[0]
            #+shift[0]
            +start_blue
            ,linestyle='--',
            c=clrs[ctr_clrs],
           label='rewarded lever pull')
for k in range(1,temp.shape[0],):
    plt.axvline(x=temp[k]
                #+shift[0]
                +start_blue
            ,linestyle='--',

                c=clrs[ctr_clrs])
ctr_clrs+=1
    
# plot rest of events
ctr=0
for k in [2,3,7]:
    temp = behavior_times[k]
    print (temp.shape)
    temp = temp/30+ start_blue
    plt.scatter(temp , 
                temp*0+1300+ ctr*50, 
                c=clrs[ctr_clrs],
                label=names_behaviors[k])
    ctr+=1
    ctr_clrs+=1
    
plt.legend()
plt.show()






# sessions:  71
names:  ['rewarded_lever_pulls' 'unrewarded_lever_pulls' 'left_paw' 'right_paw'
 'nose' 'jaw' 'right_ear' 'tongue' 'lever_DLC']
(42, 1800, 16)
(9,)
Shifts:  [4.641 4.915 4.701]
start blue:  2.5
(128, 128, 16)
(16, 40000)
0 Retrosplenial area, dorsal part, layer 1 - right
1 Retrosplenial area, lateral agranular part, layer 1 - right
2 Primary visual area, layer 1 - right
3 Primary somatosensory area, upper limb, layer 1 - right
4 Primary somatosensory area, lower limb, layer 1 - right
5 Primary somatosensory area, barrel field, layer 1 - right
6 Secondary motor area, layer 1 - right
7 Primary motor area, Layer 1 - right
8 Primary motor area, Layer 1 - left
9 Secondary motor area, layer 1 - left
10 Primary somatosensory area, barrel field, layer 1 - left
11 Primary somatosensory area, lower limb, layer 1 - left
12 Primary somatosensory area, upper limb, layer 1 - left
13 Primary visual area, layer 1 - left
14 Retrosplenial area, lateral agranular part, layer 1 - left
15 R

NameError: name 'sums' is not defined

In [52]:
##################################################
########## MAKE BEHAVIOR SEQUENCES VECTORS #######
##################################################

def get_lever_pulls_and_behaviors(session_id, root_dir, animal_id):

    # look at specific trials and neural activity
    fname = os.path.join(
        root_dir, 
        animal_id,
        'tif_files',
        session_id,
        session_id+'_body_and_nonreward_lockout_0secLockout_[]bodyfeats.npz')

    data = np.load(fname, allow_pickle=True)
    names_behaviors = data['names']
    print ("names: ", names_behaviors)
    trials = data['trials']
    print (trials.shape)
    random = data['random']
    behavior_times = data['behaviors']
    print (behavior_times.shape)
    # t = np.arange(trials.shape[1])/30.-30
    # plt.plot(t,trials[2,:,:])
    # plt.show()

    shift = np.loadtxt(os.path.join(
                                root_dir, 
                                animal_id,
                                'tif_files',
                                session_id,
                                'shift.txt'))

    print ("Shifts: ", shift)

    blue_light = np.load(os.path.join(
                                root_dir, 
                                animal_id,
                                'tif_files',
                                session_id,
                                'blue_light_frame_triggers.npz'))

    start_blue = blue_light['start_blue']/30
    print ("start blue: ", start_blue)

    fname = os.path.join(
                    root_dir, 
                    animal_id,
                    'tif_files',
                    session_id,
                    session_id+'_locanmf_wholestack.npz')

    data = np.load(fname, allow_pickle=True)

    names = data['areanames_area']
    areas = data['areas']
    names2 = data['names']
    A = data["A_reshape"]
    print (A.shape)
    whole_stack = data["whole_stack"]
    print (whole_stack.shape)

    clrs = ['pink','blue','lightblue','slategrey','black','magenta','khaki']
    ctr_clrs =0
    #

    area_ids = [12, 8, 13]  # left limb, left M1, left visual

    # rewarded lever pulls in seconds
    reward_times_sec = behavior_times[0]/30. + start_blue

    # get left paw, right paw and licking
    feature_ids = [2,3,7]
    behavior_times_sec = behavior_times[feature_ids]/30 + start_blue

    return reward_times_sec, behavior_times_sec

##########################################
root_dir = '/media/cat/4TBSSD/yuki/'
animal_id = 'IA1'

# 
sessions = Visualize.get_sessions(root_dir,
                                  animal_id,
                                  'all')
print ("# sessions: ", len(sessions))

allstacks = []
for session_id in sessions:
    
    try:
        reward_times_sec, behavior_times_sec = get_lever_pulls_and_behaviors(session_id, root_dir, animal_id)
    except:
        continue

    print (reward_times_sec)
    print (behavior_times_sec[0].shape)

    # make behavivor boolean vectors
    bin_width = 0.250 # bin width in seconds
    behavior_binned = np.zeros((int(10*1//bin_width)))
    print (behavior_binned.shape)
    names = ['left paw','right paw','licking']
    # loop over lever pulls:

    behaviors = [0,1,2]
    stacks = []
    for behavior in behaviors: 
        vector_stack = []
        for k in range(reward_times_sec.shape[0]):
            temp1 = reward_times_sec[k]
            temp_behavior = behavior_binned.copy()
            for p in range(behavior_times_sec[behavior].shape[0]):
                temp2 = behavior_times_sec[behavior][p]
                diff = temp1-temp2
                if diff<=10 and diff>0:
                    # find bin where it belongs
                    idx = int(behavior_binned.shape[0] - diff/bin_width)
                    #print (diff, idx)
                    temp_behavior[idx] =1

            vector_stack.append(temp_behavior)

        vector_stack = np.array(vector_stack)
        print (vector_stack.shape)

        #
        if False:
            clrs1 = ['Greys','Purples','copper']
            plt.figure(figsize=(10,5))
            clb = plt.imshow(vector_stack,
                             aspect='auto',
                             extent=[-10,0,0,vector_stack.shape[0]],
                             cmap=clrs1[behavior]
                            )
            plt.colorbar()
            plt.xlabel("Time (sec)")
            plt.ylabel("Trial #")

            plt.savefig('/home/cat/'+str(behavior)+'.svg')
            plt.close()

        stacks.append(vector_stack)
        
    allstacks.append(stacks)
    #plt.show()

# sessions:  71
names:  ['rewarded_lever_pulls' 'unrewarded_lever_pulls' 'left_paw' 'right_paw'
 'nose' 'jaw' 'right_ear' 'tongue' 'lever_DLC']
(25, 1800, 16)
(9,)
Shifts:  [9.372 9.404 9.074]
start blue:  2.5
(128, 128, 16)
(16, 40000)
[  62.16666667  188.9         259.5         276.13333333  331.43333333
  451.03333333  535.7         543.96666667  567.03333333  597.66666667
  624.16666667  701.26666667  708.53333333  798.6         970.53333333
  981.23333333 1048.83333333 1057.03333333 1091.5        1106.3
 1186.4        1204.33333333 1217.3        1225.1        1232.1
 1253.03333333]
(921,)
(40,)
(26, 40)
(26, 40)
(26, 40)
names:  ['rewarded_lever_pulls' 'unrewarded_lever_pulls' 'left_paw' 'right_paw'
 'nose' 'jaw' 'right_ear' 'tongue' 'lever_DLC']
(61, 1800, 16)
(9,)
Shifts:  [5.184 5.181 5.184]
start blue:  2.033333333333333
(128, 128, 16)
(16, 40000)
[  74.          138.3         196.5         236.93333333  247.56666667
  255.73333333  262.53333333  269.56666667  281.46666667  29

In [ ]:
################################
############ RUN PCA ###########
################################
print (len(allstacks), len(allstacks[0]), allstacks[0][0].shape)

clrs2 = ['black','purple','khaki']
from sklearn.decomposition import PCA
a1 = []
for p in range(3):
    
    #
    temp_array =[]
    for k in range(len(allstacks)):
        temp2 = allstacks[k][p]
        temp_array.append(temp2)
    temp_array = np.vstack(temp_array)
        
    X = temp_array
    # run pca
    pca = PCA(n_components=2)
    print ("input to pca: ", X.shape)
    X_pca = pca.fit_transform(X)
    print (" pca done: ", X_pca.shape)

    fig=plt.figure(figsize=(8,8))
    plt.scatter(X_pca[:,0],
                X_pca[:,1],
                c=clrs2[p],
                #cmap = clrs1[p],
                s=100,
               # alpha=.8
               )
    plt.savefig('/home/cat/'+str(p)+'.svg')
    plt.close()
    #plt.show()
    a1.append(X)
    
# stack all feature vecotrs
X = np.vstack(np.array(a1).transpose(0,2,1)).T
print (X.shape)

# run pca
pca = PCA(n_components=2)
print ("input to pca: ", X.shape)
X_pca = pca.fit_transform(X)
print (" pca done: ", X_pca.shape)

fig=plt.figure(figsize=(8,8))
plt.scatter(X_pca[:,0],
            X_pca[:,1],
            c='blue',
            edgecolor='black',
            #cmap = clrs1[p],
            s=50,
           # alpha=.8
           )
plt.savefig('/home/cat/all.svg')
plt.close()
#plt.show()



22 3 (26, 40)
input to pca:  (619, 40)
 pca done:  (619, 2)
input to pca:  (619, 40)
 pca done:  (619, 2)
input to pca:  (619, 40)
 pca done:  (619, 2)
(619, 120)
input to pca:  (619, 120)
 pca done:  (619, 2)
